In [42]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd

In [43]:
## cleaning text 
import nltk ,re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string 
from nltk.stem import WordNetLemmatizer
import numpy as np

In [44]:
#defining the object for Lemmatization
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Implement a Transformer block as a layer

In [45]:

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


## Implement embedding layer

Two seperate embedding layers, one for tokens, one for token index (positions).

In [46]:

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions


## prepare dataset

In [47]:
train = pd.read_csv("../input/fake-news/train.csv")
# drop all null values
train= train.dropna()

In [48]:
wordnet_lemmatizer = WordNetLemmatizer()
stopwords=stopwords.words('english')
stemmer=PorterStemmer()
# clean unwanted text like stopwords, @(Mention), https(url), #(Hashtag), punctuations
def removeUnwantedText(text):
    #remove urls
    if text == np.NaN or type(text) != str:
      text = " "
    text = re.sub(r'http\S+', " ", text)
    text = re.sub(r'@\w+',' ',text)
    text = re.sub(r'#\w+', ' ', text)
    text = re.sub('r<.*?>',' ', text)
    # html tags
    text = text.lower()
    text = text.split()
    text = " ".join([word for word in text if not word in stopwords])
    for punctuation in string.punctuation:
        text = text.replace(punctuation, "")
    return text


In [49]:
train['title']=train['title'].apply(removeUnwantedText)
max(train['title'].apply( lambda x : len(x.split(" "))))

51

In [50]:
def lemmatize_text(text):
    text= text.lower()
    text= text.split(" ")
    text = " ".join([wordnet_lemmatizer.lemmatize(word) for word in text])
    return text

In [51]:
train['title']=train['title'].apply(lemmatize_text)

In [52]:
vocab_size = 25000  # Only consider the top 20k words
maxlen = 64  # Only consider the first 200 words of each movie review

In [55]:
from tensorflow.keras.preprocessing import text
from tensorflow.keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
tokenizer = text.Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train["title"])
def prep_text(texts, tokenizer, max_sequence_length):
    # Turns text into into padded sequences.
    text_sequences = tokenizer.texts_to_sequences(texts)
    return sequence.pad_sequences(text_sequences, maxlen=max_sequence_length)



In [60]:

## coverting to matrix 
x= prep_text(train['title'],tokenizer,maxlen)
x= np.array(x)
y =np.array(train['label'])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=4)

## Create classifier model using transformer layer

Transformer layer outputs one vector for each time step of our input sequence.
Here, we take the mean across all time steps and
use a feed forward network on top of it to classify text.

In [63]:

embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)


## Train and Evaluate

In [64]:
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    x_train, y_train, batch_size=32, epochs=3, validation_data=(x_test, y_test)
)

Epoch 1/3
400/400 [==============================] - 6s 13ms/step - loss: 0.5422 - accuracy: 0.6837 - val_loss: 0.1642 - val_accuracy: 0.9313
Epoch 2/3
400/400 [==============================] - 5s 12ms/step - loss: 0.0956 - accuracy: 0.9649 - val_loss: 0.1756 - val_accuracy: 0.9360
Epoch 3/3
400/400 [==============================] - 5s 13ms/step - loss: 0.0260 - accuracy: 0.9943 - val_loss: 0.2263 - val_accuracy: 0.9304
